In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import fiona
from fiona.env import GDALEnv

import geopandas as gpd
#!pip3 install pyproj==1.9.6
import pyproj
import psycopg2
import sqlite3
import shapely.wkb as swkb
import requests
#!pip3 install geojson
import geojson

from sqlalchemy.engine.url import URL
from sqlalchemy import create_engine, event
from sqlalchemy import MetaData
from sqlalchemy.orm import sessionmaker
#!pip3 install geoalchemy2
from geoalchemy2 import WKTElement, Geometry

import os
os.getcwd()

'C:\\Users\\gilramolete\\OneDrive - UNIONBANK of the Philippines\\Documents 1\\Geospatial Analysis\\Automating GIS Processes'

# Vector Data I/O in Python

In [2]:
# See all available drivers supported by GDAL
env = GDALEnv()

# This will print all available Drivers supported by GDAL
# All these can be used for reading data from them and most often also writing data into
env.drivers().keys()

dict_keys(['PCIDSK', 'netCDF', 'PDS4', 'JP2OpenJPEG', 'PDF', 'MBTiles', 'EEDA', 'DB2ODBC', 'ESRI Shapefile', 'MapInfo File', 'UK .NTF', 'OGR_SDTS', 'S57', 'DGN', 'OGR_VRT', 'REC', 'Memory', 'BNA', 'CSV', 'NAS', 'GML', 'GPX', 'KML', 'GeoJSON', 'GeoJSONSeq', 'ESRIJSON', 'TopoJSON', 'OGR_GMT', 'GPKG', 'SQLite', 'ODBC', 'WAsP', 'PGeo', 'MSSQLSpatial', 'PostgreSQL', 'OpenFileGDB', 'XPlane', 'DXF', 'CAD', 'Geoconcept', 'GeoRSS', 'GPSTrackMaker', 'VFK', 'PGDUMP', 'OSM', 'GPSBabel', 'SUA', 'OpenAir', 'OGR_PDS', 'WFS', 'WFS3', 'HTF', 'AeronavFAA', 'Geomedia', 'EDIGEO', 'GFT', 'SVG', 'CouchDB', 'Cloudant', 'Idrisi', 'ARCGEN', 'SEGUKOOA', 'SEGY', 'XLS', 'ODS', 'XLSX', 'ElasticSearch', 'Walk', 'Carto', 'AmigoCloud', 'SXF', 'Selafin', 'JML', 'PLSCENES', 'CSW', 'VDV', 'GMLAS', 'MVT', 'TIGER', 'AVCBin', 'AVCE00', 'NGW', 'HTTP'])

In [3]:
# Available drivers in geopandas/fiona. Same as: fiona.supported_drivers
gpd.io.file.fiona.drvsupport.supported_drivers

{'AeronavFAA': 'r',
 'ARCGEN': 'r',
 'BNA': 'raw',
 'DXF': 'raw',
 'CSV': 'raw',
 'OpenFileGDB': 'r',
 'ESRIJSON': 'r',
 'ESRI Shapefile': 'raw',
 'GeoJSON': 'rw',
 'GeoJSONSeq': 'rw',
 'GPKG': 'rw',
 'GML': 'raw',
 'GPX': 'raw',
 'GPSTrackMaker': 'raw',
 'Idrisi': 'r',
 'MapInfo File': 'raw',
 'DGN': 'raw',
 'PCIDSK': 'r',
 'S57': 'r',
 'SEGY': 'r',
 'SUA': 'r',
 'TopoJSON': 'r'}

## Reading and writing Shapefiles

In [3]:
from pyproj import CRS

crs=CRS('EPSG:4326').to_proj4()

# Read file from Shapefile
fp = "data/L2_data/Europe_borders.shp"
#from pyproj import Proj, transform

data = gpd.read_file(fp)
#data.set_crs(epsg = 4326, inplace = True)
data.head()

CRSError: Invalid projection: EPSG:4326: (Internal Proj Error: proj_create: SQLite error on SELECT name, type, coordinate_system_auth_name, coordinate_system_code, datum_auth_name, datum_code, area_of_use_auth_name, area_of_use_code, text_definition, deprecated FROM geodetic_crs WHERE auth_name = ? AND code = ?: no such column: area_of_use_auth_name)

In [7]:
# Write to Shapefile (just make a copy)
outfp = "data/Europe_borders_copy.shp"
data.to_file(outfp)

NameError: name 'data' is not defined

## Reading and writing GeoDatabase

In [ ]:
# Read file from File Geodatabase
fp = "data/L2_data/Finland.gdb"
data = gpd.read_file(fp, driver = "FileGDB", layer = 'country')

# Write to same FileGDB (just add a new layer)
outfp = "L2_data/Finland.gdb"
data.to_file(outfp, driver = "FileGDB", layer = "country_copy")

## Reading and writing MapInfo Tab

In [ ]:
# Read file from MapInfo Tab
fp = "L2_data/Finland.tab"
data = gpd.read_file(fp, driver = "MapInfo File")

# Write to same FileGDB (just add a new layer)
outfp = "L2_data/Finland_copy.tab"
data.to_file(outfp, driver = "MapInfo File")

## Reading PostGIS database using psycopg2

In [ ]:
# Create connection to database with psycopg2 module (update params according your db)
conn, cursor = psycopg2.connect(dbname = 'my_postgis_database', user = 'my_username', password = 'my_pwd',
                                host = '123.22.432.16', port = 5432)

# Specify sql query
sql = "SELECT * FROM MY_TABLE;"

# Read data from PostGIS
data = gpd.read_postgis(sql = sql, con = conn)

## Reading and writing PostGIS database using SqlAlchemy + GeoAlchemy

In [ ]:
# Update with your db parameters
HOST = '123.234.345.16'
DB = 'my_database'
USER = 'my_user'
PORT = 5432
PWD = 'my_password'

# Database info
db_url = URL(drivername = 'postgresql+psycopg2', host = HOST, database = DB,
                   username = USER, port = PORT, password = PWD)

# Create engine
engine = create_engine(db_url)

# Init Metadata
meta = MetaData()

# Load table definitions from db
meta.reflect(engine)

# Create session
Session = sessionmaker(bind = engine)
session = Session()

# ========================
# Read data from PostGIS
# ========================

# Specify sql query
sql = "SELECT * FROM finland;"

# Pull the data
data = gpd.read_postgis(sql = sql, con = engine)

# Close session
session.close()

# =========================================
# Write data to PostGIS (make a copy table)
# =========================================

# Coordinate Reference System (srid)
crs = 4326

# Target table
target_table = 'finland_copy'

# Convert Shapely geometries to WKTElements into column 'geom' (default in PostGIS)
data['geom'] = data['geometry'].apply(lambda row: WKTElement(row.wkt, srid=crs))

# Drop Shapely geometries
data = data.drop('geometry', axis = b1)

# Write to PostGIS (overwrite if table exists, be careful with this! )
# Possible behavior: 'replace', 'append', 'fail'

data.to_sql(target_table, engine, if_exists = 'replace', index = False)

## Reading and writing Spatialite database

In [ ]:
# DB path
dbfp = 'L2_data/Finland.sqlite'

# Name for the table
tbl_name = 'finland'

# SRID (crs of your data)
srid = 4326

# Parse Geometry type of the input Data
gtype = data.geom_type.unique()
assert len(gtype) == 1, "Mixed Geometries! Cannot insert into SQLite table."
geom_type = gtype[0].upper()

# Initialize database engine
engine = create_engine('sqlite:///{db}'.format(db = dbfp), module=sqlite)

# Initialize table without geometries
geo = data.drop(['geometry'], axis = 1)

with sqlite3.connect(dbfp) as conn:
    geo.to_sql(tbl_name, conn, if_exists = 'replace', index = False)

# Enable spatialite extension
with sqlite3.connect(dbfp) as conn:
    conn.enable_load_extension(True)
    conn.load_extension("mod_spatialite")
    conn.execute("SELECT InitSpatialMetaData(1);")
    # Add geometry column with specified CRS with defined geometry typehaving two dimensions
    conn.execute(
        "SELECT AddGeometryColumn({table}, 'wkb_geometry',\
        {srid}, {geom_type}, 2);".format(table = tbl_name, srid = srid, geom_type = geom_type)
    )

# Convert Shapely geometries into well-known-binary format
data['geometry'] = data['geometry'].apply(lambda geom: swkb.dumps(geom))

# Push to database (overwrite if table exists)
data.to_sql(tbl_name, engine, if_exists = 'replace', index = False)

## Read Web Feature Service (WFS)

In [35]:
# Specify the url for the backend. Here we are using data from Statistics Finland: https://www.stat.fi/org/avoindata/paikkatietoaineistot_en.html
url = 'http://geo.stat.fi/geoserver/vaestoruutu/wfs'

# Specify parameters (read data in json format).
# Available feature types in this particular data source: http://geo.stat.fi/geoserver/vaestoruutu/wfs?service=wfs&version=2.0.0&request=describeFeatureType
params = dict(service = 'WFS', version = '2.0.0', request = 'GetFeature',
         typeName = 'vaestoruutu:vaki2017_5km', outputFormat = 'json')

# Fetch data from WFS using requests
r = requests.get(url, params = params)

# Create GeoDataFrame from geojson
data = gpd.GeoDataFrame.from_features(geojson.loads(r.content))
data.head()

,geometry,objectid,kunta,grd_id,id_nro,xkoord,ykoord,vaesto,miehet,naiset,ika_0_14,ika_15_64,ika_65_,bbox
0,"POLYGON ((85000.000 6690000.000, 85000.000 669...",74,043,5kmN1338E0017,2436,85000,6690000,58,25,33,7,39,12,"[85000, 6690000, 90000, 6695000]"
1,"POLYGON ((85000.000 6695000.000, 85000.000 670...",75,043,5kmN1339E0017,2571,85000,6695000,450,231,219,66,266,118,"[85000, 6695000, 90000, 6700000]"
2,"POLYGON ((85000.000 6700000.000, 85000.000 670...",76,043,5kmN1340E0017,2706,85000,6700000,80,44,36,7,56,17,"[85000, 6700000, 90000, 6705000]"
3,"POLYGON ((90000.000 6690000.000, 90000.000 669...",97,043,5kmN1338E0018,2437,90000,6690000,36,19,17,3,19,14,"[90000, 6690000, 95000, 6695000]"
4,"POLYGON ((90000.000 6695000.000, 90000.000 670...",98,043,5kmN1339E0018,2572,90000,6695000,275,139,136,47,160,68,"[90000, 6695000, 95000, 6700000]"
